In [ ]:
import json
import os
import shutil
import warnings
from datetime import datetime
import sys

import pandas as pd
import typer
from dotenv import find_dotenv, load_dotenv
from fastcore.xtras import Path

from extracao.stations import Estacoes
from extracao.format import merge_on_frequency, LIMIT_FREQ
from extracao.location import Geography


load_dotenv(find_dotenv(), override=True)
warnings.simplefilter('ignore')

SQLSERVER_PARAMS = dict(
	driver=os.environ.get('SQL_DRIVER'),
	server=os.environ.get('SQL_SERVER'),
	database=os.environ.get('SQL_DATABASE'),
	trusted_conn=True,
	mult_results=True,
	encrypt=False,
	timeout=int(os.environ.get('SQL_TIMEOUT')),
)

if sys.platform in ('linux', 'darwin', 'cygwin'):
	SQLSERVER_PARAMS.update(
		{
			'trusted_conn': False,
			'mult_results': False,
			'username': os.environ.get('USERNAME'),
			'password': os.environ.get('PASSWORD'),
		}
	)

MONGO_URI: str = os.environ.get('MONGO_URI')

In [ ]:
e = data = Estacoes(SQLSERVER_PARAMS, MONGO_URI, read_cache=True)

In [ ]:
dfs = e.extraction()

In [ ]:
dfs[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701351 entries, 0 to 701350
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Frequência            701351 non-null  string
 1   Entidade              701350 non-null  string
 2   Fistel                701351 non-null  string
 3   Serviço               701351 non-null  string
 4   Estação               701351 non-null  string
 5   Latitude              701351 non-null  string
 6   Longitude             701351 non-null  string
 7   Código_Município      701351 non-null  string
 8   Município             200360 non-null  string
 9   UF                    701337 non-null  string
 10  Classe                701351 non-null  string
 11  Classe_Emissão        701351 non-null  string
 12  Largura_Emissão(kHz)  701351 non-null  string
 13  Validade_RF           701351 non-null  string
 14  Status                701351 non-null  string
 15  Fonte            

In [ ]:
aero = dfs.pop()
anatel = pd.concat(dfs, ignore_index=True)

In [ ]:
anatel = anatel.astype('string', copy=False)

In [ ]:
anatel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012582 entries, 0 to 1012581
Data columns (total 29 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   Frequência                   1012582 non-null  string
 1   Entidade                     1012580 non-null  string
 2   Fistel                       1012582 non-null  string
 3   Serviço                      1012582 non-null  string
 4   Estação                      1011621 non-null  string
 5   Latitude                     998494 non-null   string
 6   Longitude                    998494 non-null   string
 7   Código_Município             1012582 non-null  string
 8   Município                    480943 non-null   string
 9   UF                           1012568 non-null  string
 10  Classe                       1012153 non-null  string
 11  Classe_Emissão               1012160 non-null  string
 12  Largura_Emissão(kHz)         1012160 non-null  string
 1

In [ ]:
df = merge_on_frequency(anatel, aero)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012351 entries, 0 to 1012350
Data columns (total 29 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   Frequência                   1012351 non-null  string
 1   Entidade                     1012349 non-null  string
 2   Fistel                       1010769 non-null  string
 3   Serviço                      1010769 non-null  string
 4   Estação                      1009808 non-null  string
 5   Latitude                     998263 non-null   string
 6   Longitude                    998263 non-null   string
 7   Código_Município             1010769 non-null  string
 8   Município                    479130 non-null   string
 9   UF                           1010755 non-null  string
 10  Classe                       1010341 non-null  string
 11  Classe_Emissão               1010348 non-null  string
 12  Largura_Emissão(kHz)         1010348 non-null  string
 1

In [ ]:
g = Geography(df)


In [ ]:
df[df.Latitude.isna()]

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
967735,207.0,TELEVISAO MIRANTE LTDA,50400772892,800,641202245,<NA>,<NA>,2102101,Brejo,MA,...,50.0,7.5,0,0.0,30.0,0.95,0.5,0 0 0 0 0 0 0 0 0 1...,35,http://sistemas.anatel.gov.br/se/eApp/reports/...
967738,207.0,RADIO E TV DIFUSORA DO MARANHAO LTDA,50400741660,800,682889830,<NA>,<NA>,2105427,Itinga do Maranhão,MA,...,175.0,4.35,.00,0.0,30.0,2.97,0.5,"{'0': 0.0, '5': 0.28, '10': 0.56, '15': 1.2, '...",45,http://sistemas.anatel.gov.br/se/eApp/reports/...
967740,207.0,RADIO MIRANTE DO MARANHAO LTDA,50400772388,800,641202822,<NA>,<NA>,2105989,Lajeado Novo,MA,...,25.0,7.5,0,0.0,30.0,0.95,0.5,"{'0': 0.9, '5': 0.9, '10': 0.8, '15': 0.5, '20...",40,http://sistemas.anatel.gov.br/se/eApp/reports/...
967753,207.0,SOCIEDADE RADIO E TELEVISAO ALTEROSA S. A.,50405193467,800,<NA>,<NA>,<NA>,3143104,Monte Carmelo,MG,...,-1.0,<NA>,<NA>,<NA>,<NA>,<NA>,0.5,<NA>,<NA>,http://sistemas.anatel.gov.br/se/eApp/reports/...
967755,207.0,CLUBE DE TELEVISAO DE MUZAMBINHO,50400357135,800,<NA>,<NA>,<NA>,3146107,Ouro Preto,MG,...,-1.0,<NA>,<NA>,<NA>,<NA>,<NA>,0.5,<NA>,<NA>,http://sistemas.anatel.gov.br/se/eApp/reports/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997631,84.9,RADIO CULTURA DE CAMPINAS LTDA,50445389737,230,1015189099,<NA>,<NA>,3509502,Campinas,SP,...,1500.0,3.24,5,255.0,96.0,0.58,0.5,"{'0': 2.03, '5': 2.45, '10': 3.01, '15': 3.67,...",115,http://sistemas.anatel.gov.br/se/eApp/reports/...
997637,87.1,EMPRESA BRASIL DE COMUNICACAO S.A. - EBC,50439624967,230,1014332505,<NA>,<NA>,2611606,Recife,PE,...,10000.0,6.32,0,210.0,40.0,0.612,0.5,"{'0': 4.01, '5': 4.01, '10': 4.01, '15': 4.15,...",60,http://sistemas.anatel.gov.br/se/eApp/reports/...
997638,87.1,EMPRESA BRASIL DE COMUNICACAO S.A. - EBC,50439625181,230,1014974000,<NA>,<NA>,3550308,São Paulo,SP,...,16500.0,3.8,5,110.0,93.0,0.35,0.5,"{'0': 2.33, '5': 2.27, '10': 2.2, '15': 2.13, ...",110,http://sistemas.anatel.gov.br/se/eApp/reports/...
997643,87.1,EMPRESA BRASIL DE COMUNICACAO S.A. - EBC,50444847766,230,<NA>,<NA>,<NA>,1100205,Porto Velho,RO,...,-1.0,<NA>,<NA>,0.0,0.0,<NA>,0.5,[],<NA>,http://sistemas.anatel.gov.br/se/eApp/reports/...


In [ ]:
df[g.missing['empty_coords']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 14088 entries, 967735 to 997661
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Frequência                   14088 non-null  string
 1   Entidade                     14088 non-null  string
 2   Fistel                       14088 non-null  string
 3   Serviço                      14088 non-null  string
 4   Estação                      13127 non-null  string
 5   Latitude                     0 non-null      string
 6   Longitude                    0 non-null      string
 7   Código_Município             14088 non-null  string
 8   Município                    14088 non-null  string
 9   UF                           14088 non-null  string
 10  Classe                       14086 non-null  string
 11  Classe_Emissão               14088 non-null  string
 12  Largura_Emissão(kHz)         14088 non-null  string
 13  Validade_RF                  1

In [ ]:
g.drop_rows_without_location_info()

In [ ]:
df[g.missing['empty_coords']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 14088 entries, 967735 to 997661
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Frequência                   14088 non-null  string
 1   Entidade                     14088 non-null  string
 2   Fistel                       14088 non-null  string
 3   Serviço                      14088 non-null  string
 4   Estação                      13127 non-null  string
 5   Latitude                     0 non-null      string
 6   Longitude                    0 non-null      string
 7   Código_Município             14088 non-null  string
 8   Município                    14088 non-null  string
 9   UF                           14088 non-null  string
 10  Classe                       14086 non-null  string
 11  Classe_Emissão               14088 non-null  string
 12  Largura_Emissão(kHz)         14088 non-null  string
 13  Validade_RF                  1

In [ ]:
g.df = g.merge_df_with_ibge(g.df)

In [ ]:
g.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012351 entries, 0 to 1012350
Data columns (total 33 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   Frequência                   1012351 non-null  string
 1   Entidade                     1012349 non-null  string
 2   Fistel                       1010769 non-null  string
 3   Serviço                      1010769 non-null  string
 4   Estação                      1009808 non-null  string
 5   Latitude                     998263 non-null   string
 6   Longitude                    998263 non-null   string
 7   Código_Município             1010769 non-null  string
 8   Município                    479130 non-null   string
 9   UF                           1010755 non-null  string
 10  Classe                       1010341 non-null  string
 11  Classe_Emissão               1010348 non-null  string
 12  Largura_Emissão(kHz)         1010348 non-null  string
 1

In [ ]:
g.df.Latitude_IBGE.isna().sum()

1760

In [ ]:
g.fill_missing_coords()

In [ ]:
g.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012351 entries, 0 to 1012350
Data columns (total 33 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   Frequência                   1012351 non-null  string
 1   Entidade                     1012349 non-null  string
 2   Fistel                       1010769 non-null  string
 3   Serviço                      1010769 non-null  string
 4   Estação                      1009808 non-null  string
 5   Latitude                     1012351 non-null  string
 6   Longitude                    1012351 non-null  string
 7   Código_Município             1010769 non-null  string
 8   Município                    479130 non-null   string
 9   UF                           1010755 non-null  string
 10  Classe                       1010341 non-null  string
 11  Classe_Emissão               1010348 non-null  string
 12  Largura_Emissão(kHz)         1010348 non-null  string
 1

In [ ]:
g.df.Latitude.isna().sum()

0

In [ ]:
folder = Path.cwd().parent / "extracao" / "datasources" / "arquivos" / "saida"

In [ ]:
def read_df(name: str)->pd.DataFrame:
    return pd.read_parquet(folder / f"{name}.parquet.gzip", dtype_backend="numpy_nullable")

In [ ]:
df = read_df("estacoes")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012351 entries, 0 to 1012350
Data columns (total 38 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   Frequência                   1012351 non-null  string
 1   Entidade                     1012349 non-null  string
 2   Fistel                       1010769 non-null  string
 3   Serviço                      1010769 non-null  string
 4   Estação                      1009808 non-null  string
 5   Latitude                     1010034 non-null  string
 6   Longitude                    1010034 non-null  string
 7   Código_Município             1012190 non-null  string
 8   Município                    479130 non-null   string
 9   UF                           1010755 non-null  string
 10  Classe                       1010341 non-null  string
 11  Classe_Emissão               1010348 non-null  string
 12  Largura_Emissão(kHz)         1010348 non-null  string
 1

In [ ]:
df[df.Latitude.isna()]

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Relatório_Canal,Município_IBGE,Latitude_IBGE,Longitude_IBGE,UF_IBGE,CD_MUN,NM_MUN,SIGLA_UF,LAT,LON
13564,373.3375,Compass Minerals America do Sul Industria e Co...,50419576657,019,1011999819,<NA>,<NA>,3524408,Jacarei,SP,...,<NA>,<NA>,<NA>,<NA>,<NA>,3524402,Jacareí,SP,-23.273746,-46.05695
13565,373.3375,Compass Minerals America do Sul Industria e Co...,50419576657,019,1011999819,<NA>,<NA>,3524408,Jacarei,SP,...,<NA>,<NA>,<NA>,<NA>,<NA>,3524402,Jacareí,SP,-23.273746,-46.05695
13725,373.3875,Basf S.A.,50443956332,019,1014613253,<NA>,<NA>,2607901,Jaboatão dos Guararapes,PE,...,<NA>,<NA>,<NA>,<NA>,<NA>,2607901,Jaboatão dos Guararapes,PE,-8.181873,-34.94861
13726,373.3875,Basf S.A.,50443956332,019,1014613261,<NA>,<NA>,2607901,Jaboatão dos Guararapes,PE,...,<NA>,<NA>,<NA>,<NA>,<NA>,2607901,Jaboatão dos Guararapes,PE,-8.181661,-34.948703
13727,373.3875,Basf S.A.,50443956332,019,1014613270,<NA>,<NA>,2607901,Jaboatão dos Guararapes,PE,...,<NA>,<NA>,<NA>,<NA>,<NA>,2607901,Jaboatão dos Guararapes,PE,-8.181873,-34.94861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997657,87.1,EMPRESA BRASIL DE COMUNICACAO S.A. - EBC,50447054627,230,[],<NA>,<NA>,5006200,Nova Andradina,MS,...,http://sistemas.anatel.gov.br/se/eApp/reports/...,Nova Andradina,-22.238,-53.3437,50,<NA>,<NA>,<NA>,<NA>,<NA>
997658,79.1,RADIO EDUCADORA LTDA,50446499293,230,[],<NA>,<NA>,4304606,Canoas,RS,...,http://sistemas.anatel.gov.br/se/eApp/reports/...,Canoas,-29.9128,-51.1857,43,<NA>,<NA>,<NA>,<NA>,<NA>
997659,84.3,MINISTERIO DA DEFESA,50441480594,230,[],<NA>,<NA>,1501402,Belém,PA,...,http://sistemas.anatel.gov.br/se/eApp/reports/...,Belém,-1.4554,-48.4898,15,<NA>,<NA>,<NA>,<NA>,<NA>
997663,86.5,RADIO EDUCADORA DE PIRACICABA LTDA,50445555416,230,[],<NA>,<NA>,3538709,Piracicaba,SP,...,http://sistemas.anatel.gov.br/se/eApp/reports/...,Piracicaba,-22.7338,-47.6476,35,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
m = pd.read_csv( folder.parent / 'municipios.csv', dtype='string', dtype_backend='numpy_nullable')

In [ ]:
m[m['Código_Município'] == '3554102']

,Código_Município,Município,Latitude,Longitude,Capital,UF
5185,3554102,Taubaté,-23.0104,-45.5593,0,35


In [ ]:
df.loc[df["Código_Município"] != df["CD_MUN"], ["Código_Município", "NM_MUN", "CD_MUN", "SIGLA_UF", 'LAT', 'LON', 'Latitude_IBGE', 'Longitude_IBGE']].sample(10)

ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
df[df.Latitude.isna()]

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Município_IBGE,Latitude_IBGE,Longitude_IBGE,UF_IBGE,geometry,CD_MUN,NM_MUN,SIGLA_UF,LAT,LON
194,14767.0,TELEFONICA BRASIL S.A.,50417179405,019,1013196128,<NA>,<NA>,3550308,São Paulo,SP,...,São Paulo,-23.5329,-46.6395,35,"b""\x01\x01\x00\x00\x00\xdc\xb7Z'.7G\xc0\x1f\xf...",3528502,Mairiporã,SP,-23.313174,-46.431096
197,168.90625,POLICIA MILITAR DO ESTADO DE SÃO PAULO,50403450381,019,1001183930,<NA>,<NA>,3516408,<NA>,SP,...,Franco da Rocha,-23.3229,-46.729,35,b'\x01\x01\x00\x00\x00\n\x14\xb1\x88aKG\xc0A\x...,3528502,Mairiporã,SP,-23.322453,-46.588914
202,168.39375,POLICIA MILITAR DO ESTADO DE SÃO PAULO,50403450381,019,1001183930,<NA>,<NA>,3516408,<NA>,SP,...,Franco da Rocha,-23.3229,-46.729,35,b'\x01\x01\x00\x00\x00\n\x14\xb1\x88aKG\xc0A\x...,3528502,Mairiporã,SP,-23.322453,-46.588914
215,165.35,PROSEGUR BRASIL S/A TRANSPORTADORA DE VALORES ...,50419608281,019,699064074,<NA>,<NA>,3550308,<NA>,SP,...,São Paulo,-23.5329,-46.6395,35,b'\x01\x01\x00\x00\x00\xc0@\x10 CUG\xc0\xecQ\x...,3528502,Mairiporã,SP,-23.42,-46.666111
218,166.95625,POLICIA MILITAR DO ESTADO DE SÃO PAULO,50403450381,019,1001183930,<NA>,<NA>,3516408,<NA>,SP,...,Franco da Rocha,-23.3229,-46.729,35,b'\x01\x01\x00\x00\x00\n\x14\xb1\x88aKG\xc0A\x...,3528502,Mairiporã,SP,-23.322453,-46.588914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975278,17823.75,TELEFONICA BRASIL S.A.,50417179405,019,1013196209,<NA>,<NA>,3501905,Amparo,SP,...,Amparo,-22.7088,-46.772,35,b'\x01\x01\x00\x00\x00\xca\x17\xb4\x90\x80WG\x...,3531209,Monte Alegre do Sul,SP,-22.691944,-46.683611
975281,14.77400001,CLARO S.A.,50013414186,046,691959390,<NA>,<NA>,3501905,Amparo,SP,...,Amparo,-22.7088,-46.772,35,b'\x01\x01\x00\x00\x00\xcd\x85\xa4)\x15WG\xc0U...,3531209,Monte Alegre do Sul,SP,-22.74202777777767,-46.68033333333333
975282,14770.5,CLARO S.A.,11021017965,175,641189664,<NA>,<NA>,3501905,<NA>,SP,...,Amparo,-22.7088,-46.772,35,b'\x01\x01\x00\x00\x00\x1a\x18yY\x13WG\xc0\xa3...,3531209,Monte Alegre do Sul,SP,-22.741944,-46.680278
975283,37621.5,CLARO S.A.,11021017965,175,641189664,<NA>,<NA>,3501905,<NA>,SP,...,Amparo,-22.7088,-46.772,35,b'\x01\x01\x00\x00\x00\x1a\x18yY\x13WG\xc0\xa3...,3531209,Monte Alegre do Sul,SP,-22.741944,-46.680278
